## PoC парсера сайта decor-of-today.com

Продукт тестировался на одной категории  - светильники. Точность работы на других категориях не гарантирую.
Поэтому не промышленный релиз, а PoC.
Парсер был зарелизен примерно за 5 часов по отработанной технологии из готовых блоков. Это начиная с исследования
самого сайта и заканчивая финальной выгрузкой в датафрейм. Ну основное время, понятно, ушло на ковыряние с тегами
и варку супа. В это раз не стал заморачиваться экспериментами с сохраненим в SQL-базу и эксепшенами. 
Просто взял и быстро написал.

In [ ]:
#начинаем с импорта библиотек
import requests
from requests.auth import HTTPProxyAuth # для проксей
from  bs4 import BeautifulSoup as bs
import csv
from urllib.request import urljoin
from tqdm.notebook import tqdm
from collections import deque
import pandas as pd

In [2]:
# Задаем глобальные параметры
headers = {'accept': '*/*',
           'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.66 Safari/537.36'
          }
domen = 'https://decor-of-today.com'
url='https://decor-of-today.com/catalog/'
filename = 'd-o-t_product_links.csv'

In [76]:
#формируем список продуктовых категорий и скармливаем его пагинатору
temp_list=[]
prodact_cat = get_category_links(headers, domen, url)
for i in prodact_cat:
    a = get_pagination(i, headers) #запуск пагинатора.на одной итерации обрабатывается одна категория
    temp_list.append(a)
#выпрямляем список пагинационных урлов    
grid_links = []
for sublist in temp_list:
    for item in sublist:
        grid_links.append(item)
#сбор продуктовых урлов с грида        
temp_plinks=[]
for gl in grid_links:
    gpl = get_product_links(gl, headers, domen)
    temp_plinks.append(gpl)
#выпрямляем список продуктовых урлов    
product_links=[]
for sublist in temp_plinks:
    for item in sublist:
        product_links.append(item)   
save_products(product_links)
print('DONE!')

DONE!


In [8]:
# Функция сбора ссылок на основные продуктовые категории
def get_category_links(headers, domen, url):
    session = requests.Session()
    r = session.get(url, headers=headers)
    soup = bs(r.text, 'lxml')
    links = []
    urls = set()
    ul = soup.find('ul', class_='bx_catalog_tile_ul').find_all('a', href=True)
    for u in ul:
        urls.add(u['href'])
    for i in urls:
        link  = urljoin(domen,i)
        links.append(link)
    return links

In [7]:
#  блок сбора урлов пагинации
def get_pagination(url, headers):
    session = requests.Session()
    r = session.get(url, headers=headers)
    pagi=[] 
    if r.status_code == 200:
        soup = bs(r.content, 'lxml')
        u = str(url)+f'?SHOWALL_2=1'
        if u not in pagi:
            pagi.append(u)
    return pagi

In [6]:
# Функция сбора ссылок на целевые страницы
def get_product_links(url, headers, domen):
    session = requests.Session()
    r = session.get(url, headers=headers)
    soup = bs(r.text, 'lxml')
    links = []
    urls=set()
    if url:
        ul = soup.find('div', attrs={'id': 'catalogList'}).find_all('a', href=True)
        if ul:
            for u in ul:
                urls.add(u['href'])
            for i in urls:
                link  = urljoin(domen,i)
                links.append(link)
    return links

In [3]:
# Функция сохранения файла со ссылками на целевые страницы
def save_products (result):
    with open(filename, 'w', encoding = 'utf-8') as outfile:
        spamwriter = csv.writer(outfile, lineterminator='\n')
        spamwriter.writerow(['Product_Link']) 
        for i in result:
            spamwriter.writerow([i])
            
# Функция загрузки файла со ссылками на целевые страницы
def open_csv(filename):
    results=[]
    with open(filename, 'r') as data:
        data.readline().rstrip()
        for line in data:
            results.append(line[:-1])
    return results

#Функция сохранения результатов парсинга целевых страниц
def save_product_info(products):
    with open('c:/Users/Vladimir/Coding/Notebooks/Парсинг/Проекты/decor-of-today/products.csv', 'w', encoding = 'utf-8') as outfile:
        spamwriter = csv.DictWriter(outfile, fieldnames=get_field_name(products), lineterminator='\n')    
        spamwriter.writeheader() #записывет заголовки столбцов равные ключам словаря
        spamwriter.writerows(products)
        
# Вспом.функция для сбора названия полей с словарях с продуктами для формирования заголовка в CSV (Без сортировки)
def get_field_name(products):
    field_names = set()
    for i in products:
        for k in i:
            field_names.add(k)
    return field_names

#функция проверки уже спарсенных урлов (чтобы не запускать по ним парсинг повторно)
def get_visited_urls():
    try:
        file = 'products.csv'
        if file:
            df = pd.read_csv(file, encoding = 'utf-8')
            urls_done = df['5url'].tolist()
            visited = set(urls_done)
            return visited, df
    except FileNotFoundError:
        df=pd.DataFrame()
        visited=set()
        return visited, df # если файла не существует, товернем путой сет и список

# Функция объединения прошлых и текущих результатов в 1 общий файл    
def add_previous_part(df):
    if df.shape[0] != 0:
        file = 'products.csv'
        df.to_csv(file, mode='a', index=False, encoding = 'utf-8', header=False)
    else:
        pass   

In [4]:
# ВЕРСИЯ 2 С ПРОМЕЖУТОЧНЫМ СОХРАНЕНИЕМ
def get_info2(headers, filename):
    global df
    results = open_csv(filename)
    queue = deque()
    queue.extend(results)
    visited = get_visited_urls()[0]
    df = get_visited_urls()[1]
    session = requests.Session()
    products = [] 
    while tqdm(queue, desc='Getting info from product page url', leave=False):   
        url = queue.popleft()
        if url in visited:
            continue # начнет цикл while заново     
        params={} # словарь для занесения параметров//можно в конце цикла все добавить как в кинопоиске без создания словаря
        r = session.get(url, headers=headers)
        soup = bs(r.text, 'lxml')
        if soup:
            visited.add(url) #добавляем в посещенные, только когда получим суп. т.е. страница доступна
            try:
                a=soup.find('h1', itemprop='name').text
                if a:
                    params['1_title']=a
                b=soup.find('div', class_='article').text
                if b:
                    params['2_articul']=b
                c=soup.find('div', class_='price').text
                if c:
                    params['3_price']=c
                d=soup.find('div', class_='item-stock').text
                if d:
                    params['4_stock']=d
                params['5url']=url
                f=soup.find('img', class_='img-responsive zoommed')['src']
                params['6img']=str(domen)+str(f)
                k1=[]
                k2=[]
                dict1={}
                x=soup.find('div', class_='bottom-props-content bottom-props_props active').find_all('div', class_='bottom-props_props-name')
                for i in x:
                    k1.append(i.text)
                y=soup.find('div', class_='bottom-props-content bottom-props_props active').find_all('div', class_='bottom-props_props-val')
                for i in y:
                    k2.append(i.text)
                dict1 = dict(zip(k1, k2))
                params.update(dict1)
                products.append(params)
            except:
                products.append(params)
                print('Часть тегов не обнаружена. Двигаюсь дальше')
                pass
        else:
            print('Проблемы с супом')
            save_product_info(products) # промежуточное сохранение на случай если суп не обнаружен
            return False
    save_product_info(products) #  для сохранения в CSV
    add_previous_part(df) # перезапись предыдущих результатов в случае перезапуска парсера
    print('DONE!')

In [13]:
get_info2(headers, filename)

DONE!


In [14]:
import pandas as pd
df=pd.read_csv('products.csv', encoding='utf-8')
df.sort_index(inplace=True, axis=1)
df.head()

,1_title,2_articul,3_price,4_stock,5url,6img,Гарантия,Диаметр,Количество ламп,Мощность ламп (W),Напряжение (V),Производитель,Степень защиты,Тип лампочки,Тип цоколя
0,Светильник подвесной KH01107,арт. KH01107,10 500,В наличии,https://decor-of-today.com/catalog/svet/svetil...,https://decor-of-today.com/upload/resize_cache...,1 год,36 см,1.0,25,220,Индия,IP 20,Накаливания,E 27
1,Светильник подвесной KH01132,арт. KH01132,8 000,В наличии,https://decor-of-today.com/catalog/svet/svetil...,https://decor-of-today.com/upload/resize_cache...,1 год,22 см,1.0,25,220,Индия,IP 20,Накаливания,E 27
2,Светильник настенный 01537,арт. 01537,29 400,В наличии,https://decor-of-today.com/catalog/svet/svetil...,https://decor-of-today.com/upload/resize_cache...,1 год,11,1.0,40,220,Индия,IP 54,Накаливания,E 27
3,Светильник настенный 01551,арт. 01551,14 500,В наличии,https://decor-of-today.com/catalog/svet/svetil...,https://decor-of-today.com/upload/resize_cache...,1 год,16,1.0,25,220,Индия,IP 20,Накаливания,E 14
4,Светильник настенный 01547,арт. 01547,12 800,В наличии,https://decor-of-today.com/catalog/svet/svetil...,https://decor-of-today.com/upload/resize_cache...,1 год,10,1.0,40,220,Индия,IP 54,Накаливания,E 27
